In [ ]:
import osmnx as ox
# get the building footprints in Macclesfield
place = "Landkreis Breisgau-Hochschwarzwald"
#tags = {'building': 'school'}
# Ebene Landkreis
tags = {'admin_level': '6'}
#gdf = ox.geometries_from_place(place, tags)

#tags = {'boundary': 'administrative'}   

bh_osmid='R1946367'
lk_bh=ox.geocode_to_gdf(bh_osmid, by_osmid=True)
lk_bh_poly=lk_bh.geometry[0]

In [ ]:
lk_bh.plot()

In [ ]:
# Ebene Gemeinde
tags = {'admin_level': '8'}
gdf =ox.features.features_from_polygon(lk_bh_poly,tags)

In [ ]:
in_bh=gdf.within(lk_bh_poly)

In [ ]:
gem_in_bh_gdf=gdf[in_bh & (gdf['name'].notnull()) & (gdf['type'] == 'boundary') ]  

In [ ]:
import json
with open('schulen.json') as f:
    x = json.load(f)

#sc=json.read_file('schulen.json')
#x = json.loads(sc)

In [ ]:
#in_bh_gdf.plot()
tags = {'boundary': 'administrative'}
gem_bh_gdf =ox.features.features_from_polygon(lk_bh_poly,tags)

In [ ]:
in_bh=gem_bh_gdf.within(lk_bh_poly)
ort_in_bh_gdf=gem_bh_gdf[in_bh & (gem_bh_gdf['name'].notnull()) & (gem_bh_gdf['type'] == 'boundary') & gem_bh_gdf['de:regionalschluessel'].isnull()] 


In [ ]:
ort_in_bh_gdf

In [ ]:
network_type = "walk"
trip_times = [5, 10, 15, 20, 25,30,40]  # in minutes
travel_speed = 3  # walking speed in km/hour
tags = {'building': True}   

# get one color for each isochrone
iso_colors = ox.plot.get_colors(n=len(trip_times), cmap="plasma", start=0)

lk_bh_walk=ox.graph_from_polygon(lk_bh_poly,network_type=network_type)
lk_bh_build=ox.features_from_polygon(lk_bh_poly,tags)

# add an edge attribute for time in minutes required to traverse each edge
meters_per_minute = travel_speed * 1000 / 60  # km per hour to m per minute
for _, _, _, data in lk_bh_walk.edges(data=True, keys=True):
    data["time"] = data["length"] / meters_per_minute

In [ ]:
lk_bh_walk.size()

In [ ]:
lk_bh_walk[456383]

In [ ]:
eki_buildings=ox.truncate.truncate_graph_polygon(lk_bh_build,poly)
figure, axes = ox.plot_graph(
    G,
    ax=axe,
    node_color=nc,
    node_size=ns,
    node_alpha=0.8,
    edge_linewidth=0.2,
    edge_color="#999999",
    show=False, 
    close=False
)

In [ ]:
import networkx as nx
from shapely.geometry import Point

br=2
anz=25
fig, axs = plt.subplots(anz,br,figsize=(30, 120))
sw=0

#roads=lk_bh_walk
for index, row in gem_in_bh_gdf.iterrows():
    node_colors = {}
    poly_gem = gpd.GeoSeries(row['geometry'])
    axs[sw//br,sw%br].set_title(row['name'])

    for cur in (x["schools"]):
        p=Point(cur["lng"],cur["lat"])
        if p.within(poly_gem).any(): 
            poi=ox.distance.nearest_nodes(roads, cur["lng"],cur["lat"])
            for trip_time, color in zip(sorted(trip_times, reverse=True), iso_colors):
                subgraph = nx.ego_graph(roads, poi, radius=trip_time, distance="time")
                for node in subgraph.nodes():
                    col_old=0
                    if node in node_colors:
                        col_old=sum(node_colors[node])
                    if sum(color)>col_old:
                        node_colors[node] = color
    
    for idx_orte, row_orte in ort_in_bh_gdf.iterrows():
        poly_ort=row_orte['geometry']
        poly_ort = gpd.GeoSeries(row_orte['geometry'])
        p_center=poly_ort.centroid
        if poly_gem.contains(p_center).any():
            poly_ort.plot(ax=axs[sw//br,sw%br],facecolor='none', edgecolor='green')
            axs[sw//br,sw%br].annotate(text=row_orte["name"], xy=(p_center[0].x,p_center[0].y), horizontalalignment='center')
    lk_bh_build.plot(ax=axs[sw//br,sw%br],color='brown')
    nc = [node_colors[node] if node in node_colors else "none" for node in roads.nodes()]
    ns = [15 if node in node_colors else 0 for node in roads.nodes()]
    poly_gem.plot(ax=axs[sw//br,sw%br],facecolor='none', edgecolor='green')
    fig, ax = ox.plot_graph(
        roads,
        ax=axs[sw//br,sw%br],
        node_color=nc,
        node_size=ns,
        node_alpha=0.8,
        edge_linewidth=0.2,
        edge_color="#999999",
        show=False, 
        close=False
    )
    
    axs[sw//br,sw%br].set_ylim(poly_gem.bounds["miny"][0],poly_gem.bounds["maxy"][0])
    axs[sw//br,sw%br].set_xlim(poly_gem.bounds["minx"][0],poly_gem.bounds["maxx"][0])
    sw+=1
    if sw>3:
        break
plt.show()

In [ ]:
import matplotlib.pyplot as plt
import geopandas as gpd
import numpy as np
import shapely
from shapely import LineString, MultiPoint, Polygon, Point, MultiPolygon
br=2
anz=25
fig, axs = plt.subplots(anz,br,figsize=(30, 120))
sw=0
for index, row in gem_in_bh_gdf.iterrows():
    roads=lk_bh_walk
    if row['name'] == 'Ehrenkirchen':
        axs[sw//br,sw%br].set_title(row['name'])
        poly = gpd.GeoSeries(row['geometry'])
        #for idx_build, row_build in lk_bh_build.iterrows():
        #    poly_build=gpd.GeoSeries(row_build['geometry'])
        #    if poly.contains(poly_build.centroid).any():
        #        poly_build.plot(ax=axs[sw//br,sw%br],color='brown')
        #for idx_orte, row_orte in ort_in_bh_gdf.iterrows():
        #    poly_ort=row_orte['geometry']
        #    poly_ort = gpd.GeoSeries(row_orte['geometry'])
        #    p_center=poly_ort.centroid
        #    if poly.contains(p_center).any():
        #        poly_ort.plot(ax=axs[sw//br,sw%br],color='white', edgecolor='green')
        #        axs[sw//br,sw%br].annotate(text=row_orte["name"], xy=(p_center[0].x,p_center[0].y), horizontalalignment='center')
        
        roads=ox.truncate.truncate_graph_polygon(lk_bh_walk,poly)
        fig, ax = ox.plot_graph(
            roads,
            ax=axs[sw//br,sw%br],
            node_color=nc,
            node_size=ns,
            node_alpha=0.8,
            edge_linewidth=0.2,
            edge_color="#999999",
            show=False, 
            close=False
        )        

        
        poly.plot(ax=axs[sw//br,sw%br],facecolor='none', edgecolor='black')
        
        sw+=1
plt.show()    

In [ ]:
type(lk_bh_build)

In [ ]:
type(node_colors)

In [ ]:
dir(p_center)

In [ ]:
p_center[0].x

In [ ]:
type(roads)

In [ ]:
#roads=ox.truncate.truncate_graph_bbox(lk_bh_walk,bbox=(poly.bounds["minx"][0],poly.bounds["miny"][0],poly.bounds["maxx"][0],poly.bounds["maxy"][0]),truncate_by_edge=False,retain_all=True)
#roads=ox.truncate.truncate_graph_bbox(lk_bh_walk,bbox=(7.4,48,7.8,48.1),truncate_by_edge=False,retain_all=True)
roads=ox.truncate.truncate_graph_polygon(lk_bh_walk,poly)
node_colors = {}
nc = [node_colors[node] if node in node_colors else "none" for node in roads.nodes()]
ns = [15 if node in node_colors else 0 for node in roads.nodes()]
fig, ax = ox.plot_graph(
    roads,
    node_color=nc,
    node_size=ns,
    node_alpha=0.8,
    edge_linewidth=0.2,
    edge_color="#999999",
    show=False, 
    close=False
)

In [ ]:
(roads.size(),lk_bh_walk.size())

In [ ]:
(poly.bounds["minx"][0],poly.bounds["miny"][0],poly.bounds["maxx"][0],poly.bounds["maxy"][0])

In [ ]:
build_tr=lk_bh_build.within(poly)

In [ ]:
lk_bh_build